In [ ]:
from _linear import LinearRegression
import _dgp as dg
from _metrics import mse_score, CrossValidation, accuracy_score,f1_score
from  _logistic import LogisticRegression
import numpy as np
import pandas as pd

<h1>Linear Regression</h1>

Generate data with no correlation

In [14]:
x,y=dg.gen(type=1)
print(x.shape)
print(y.shape)

(500, 50)
(500,)


Run linear regression and get B parameter

In [15]:
#use Newton Raphson optimisation 
regression=LinearRegression(solver="nr")
regression.fit(x,y)

algorithm did  converge under 100 iterations (at 2 iterations)


array([-1.54610666e-03,  2.50118153e+00,  1.30442515e+00,  4.94569746e-01,
       -5.02267827e-01, -3.40271557e+00, -4.28763509e-03,  7.60970653e-03,
        5.21083988e-03, -4.16208206e-03,  2.77107726e-03, -2.86986967e-03,
        7.59389124e-03,  5.09367289e-03, -9.72352090e-03, -4.21148504e-03,
       -1.15902669e-02, -5.39156069e-03, -4.74412516e-03, -4.71390234e-03,
       -4.87268331e-03, -7.10771727e-03,  7.04622571e-04, -3.80329736e-03,
        2.98162939e-03,  5.76267043e-03, -1.04852002e-02, -4.74470847e-04,
        8.93039905e-04, -6.98185435e-03,  7.63063842e-03, -2.32536805e-03,
        2.94507772e-04,  8.15449914e-04, -8.17817427e-04,  1.69555178e-03,
        6.74059199e-03,  1.02957186e-03,  1.88057473e-03,  9.69462473e-04,
        8.51456057e-04, -1.41559548e-04, -5.12563899e-03,  3.70624262e-03,
        5.09721425e-03, -3.21873693e-03,  9.41324776e-03, -4.02956861e-03,
        6.69005558e-03,  2.55924501e-03, -1.64871898e-03])

As we used NR optimisations, it converged very fast, only in 2 iterations

if We want to interpret Linear regression results, we can do so by typing this command:

In [13]:
regression.get_inference().head()

,params,std,t value,p value
0,0.000072,0.004440,0.016157,0.9871
1,2.505652,0.004386,571.248490,0.0000
2,1.297932,0.004277,303.479134,0.0000
3,0.498344,0.004082,122.087738,0.0000
4,-0.493094,0.004225,-116.715758,0.0000


We can evaluate model performance by looking at MSE

In [17]:
mse_score(y,regression.predict(x))

0.007461950038721623

The best way to evaluate model performance remains Cross Validation

In [22]:
list_of_mse=CrossValidation(Class_algorithm=regression,x=x,y=y,metrics_function=mse_score,nb_k_fold=6)
print(list_of_mse)

algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
[0.008366650090492278, 0.008926335388529164, 0.009450567718380496, 0.011295040543736782, 0.009873535736372823, 0.009144819190386013, 0.0031046842380932516]


we can average these results to get average model performance

In [23]:
np.mean(list_of_mse)

0.008594518986570114

<h1>Logistic Regression</h1>

Load data and explore 

In [34]:
table=pd.read_csv("data/exampleLR.csv")

In [35]:
table

,Unnamed: 0,V1,V2,V3,V4,V5,y
0,0,-0.560476,-0.601893,-0.995799,-0.820987,-0.511604,0
1,1,-0.230177,-0.993699,-1.039955,-0.307257,0.236938,0
2,2,1.558708,1.026785,-0.017980,-0.902098,-0.541589,1
3,3,0.070508,0.751061,-0.132175,0.627069,1.219228,0
4,4,0.129288,-1.509167,-2.549343,1.120355,0.174136,0
...,...,...,...,...,...,...,...
495,495,-1.358079,-0.089975,0.214479,0.076644,0.726564,1
496,496,0.181847,1.070516,0.892571,0.255165,1.440870,1
497,497,0.164841,-1.351100,1.018758,0.277447,-0.210170,0
498,498,0.364115,-0.522617,1.089112,0.536856,1.451281,0


Always convert data to numpy array

In [60]:
x,y=np.array(table.drop(["y","Unnamed: 0"],axis=1)),np.array(table["y"])

see if classes are balanced , in this case they are, so use of accuracy is enough

In [81]:
table["y"].value_counts()

y
0    253
1    247
Name: count, dtype: int64

Try Gradient Descent algorithm

In [61]:
model=LogisticRegression(solver="gd")
model.fit(x,y)

algorithm did not converge under 100 iterations,so the calculated parameter is biased


array([-0.1248317 , -0.98031107,  1.93586459,  0.89773156,  0.7761705 ,
       -1.07133784])

Algorithm could not converge, so the estimated parameter is biased, so try other hyperparameters to aim convergence

In [67]:
model1=LogisticRegression(solver="gd",learning_rate=0.004,max_iteration=300)
model1.fit(x,y)

algorithm did  converge under 300 iterations (at 79 iterations)


array([-0.14211795, -1.10847242,  2.18459166,  1.02655283,  0.87139786,
       -1.20943036])

Try stochastic gradient descent with 68 as size of mini batch to see the performance

In [68]:
model2=LogisticRegression(solver="sgd",learning_rate=0.001,max_iteration=300,mini_batch_size=68,add_intercept=False)
model2.fit(x,y)

algorithm did  converge under 2400 iterations (at 1080 iterations)


array([-1.03254695,  2.03007399,  0.9448015 ,  0.80938114, -1.11919829])

With new learning rate algorithm converged so we will use results of model1 to make predictions

In [72]:
model1.predict(x)[0:10]

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 1])

as well as get parameters significance

In [76]:
print(model1.get_inference())
print(model1.criteria)

     params       std   t value  p value
0 -0.142118  0.133005 -1.068517   0.2858
1 -1.108472  0.159209 -6.962360   0.0000
2  2.184592  0.222661  9.811274   0.0000
3  1.026553  0.163791  6.267456   0.0000
4  0.871398  0.144580  6.027095   0.0000
5 -1.209430  0.173253 -6.980720   0.0000
{'LL': -177.40450034974413, 'AIC_ll': 366.80900069948825, 'BIC_ll': 392.0966492900214}


Evaluate model with accuracy and f1score 

In [79]:
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=accuracy_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 122 iterations)
algorithm did  converge under 300 iterations (at 86 iterations)
algorithm did  converge under 300 iterations (at 101 iterations)
algorithm did  converge under 300 iterations (at 95 iterations)


[0.8, 0.904, 0.816, 0.864]

In [80]:
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=f1_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 114 iterations)
algorithm did  converge under 300 iterations (at 94 iterations)
algorithm did  converge under 300 iterations (at 99 iterations)
algorithm did  converge under 300 iterations (at 96 iterations)


[0.7839999999999999, 0.8925619834710744, 0.832, 0.8480000000000001]

<h1>Logistic Regression OVR

Load data

In [85]:
table=pd.read_csv("data/exampleLRMulti.csv")

Get rid of unnamed columns

In [87]:
table.drop(table.columns[table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

One more time classes are distributed equally, so we can estimate model performance with accuracy score

In [91]:
table["column_Y"].value_counts()

column_Y
1    336
0    333
2    331
Name: count, dtype: int64

In [92]:
x,y=np.array(table.drop(["column_Y"],axis=1)),np.array(table["column_Y"])

Estimate Multiclass Logistic Regression with One vs Rest algorithm and Newton Raphson optimisation 

Since it's an OVR algorithm, it will run model 3 times, this is why we have 3 converges in a row and 3 columns of B parameter

In [93]:
modelOVR=LogisticRegression(solver="nr",multiclass="ovr")
modelOVR.fit(x,y)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)


array([[-1.01498187, -1.45620846, -0.78718645],
       [ 0.34446018,  0.56751748, -0.79115877],
       [ 0.22977187, -0.65556071,  0.56300868],
       [ 0.05302252, -0.03242775, -0.03517929],
       [ 0.06803143,  0.15324563, -0.1736295 ],
       [-0.44905806,  0.79972167, -0.42289584],
       [ 0.02949896, -0.09727438,  0.07434896],
       [-0.64703364,  0.23914131,  0.39520493],
       [-0.03647586,  0.15863365, -0.09606587],
       [ 0.6808661 , -0.45149573, -0.23001672],
       [-0.01179362, -0.05067621,  0.03836354]])

We can also predict. Predictions are made always in the ascending order: 

    -first column will be 0 class
    -second column will be 1 class
    -third column will be 3 class


In [100]:
predictionsOVR=modelOVR.predict(x)
predictionsOVR 

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

We will compare these predictions to true one hot encoded labels

In [101]:
modelOVR.true_labels_matrix

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

Of course, we can always get probability (for OVR it is not normalised, but for predicted values it is normalised!!)

In [102]:
modelOVR.proba

array([[0.14660444, 0.94592762, 0.01074276],
       [0.26867955, 0.36961135, 0.19349834],
       [0.0519826 , 0.32131164, 0.59426132],
       ...,
       [0.02941952, 0.15974979, 0.77393711],
       [0.26880729, 0.33355668, 0.22603021],
       [0.03407864, 0.33402281, 0.64490065]])

We can evaluate accuracy

In [104]:
accuracy_score(predictionsOVR, modelOVR.true_labels_matrix)

0.695

But we want more intelligent evaluation, so we use cross validation 

In [107]:
acc_list=CrossValidation(Class_algorithm=modelOVR,x=x,y=modelOVR.true_labels_matrix,metrics_function=accuracy_score,nb_k_fold=8)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iter

By averaging we get more "robust" accuracy 

In [108]:
print(acc_list)
np.mean(acc_list)

[0.68, 0.632, 0.68, 0.704, 0.696, 0.696, 0.624, 0.672]


0.6729999999999999

<h1>Logistic Regression Softmax</h1>

Now, let's see another way of solving multicase without running P independent classifiers-> logistic softmax!

In [115]:
modelSFT=LogisticRegression(solver="gd",multiclass="softmax")#dont use nr for the moment
modelSFT.fit(x,y)

algorithm did  converge under 100 iterations (at 50 iterations)


array([[ 0.04194506, -0.27091767,  0.22897262],
       [ 0.23740402,  0.34859258, -0.58599659],
       [ 0.08908508, -0.4494452 ,  0.36036012],
       [ 0.0417117 , -0.02254157, -0.01917013],
       [ 0.03704339,  0.08616513, -0.12320852],
       [-0.27315999,  0.56539992, -0.29223993],
       [ 0.01838198, -0.07586092,  0.05747895],
       [-0.47306334,  0.21607495,  0.2569884 ],
       [-0.03294754,  0.10874983, -0.07580229],
       [ 0.48652451, -0.33256412, -0.15396038],
       [-0.00460555, -0.02954451,  0.03415006]])

We can compare them to modelOVR parameters

In [112]:
modelOVR.params

array([[-1.01498187, -1.45620846, -0.78718645],
       [ 0.34446018,  0.56751748, -0.79115877],
       [ 0.22977187, -0.65556071,  0.56300868],
       [ 0.05302252, -0.03242775, -0.03517929],
       [ 0.06803143,  0.15324563, -0.1736295 ],
       [-0.44905806,  0.79972167, -0.42289584],
       [ 0.02949896, -0.09727438,  0.07434896],
       [-0.64703364,  0.23914131,  0.39520493],
       [-0.03647586,  0.15863365, -0.09606587],
       [ 0.6808661 , -0.45149573, -0.23001672],
       [-0.01179362, -0.05067621,  0.03836354]])

also predict

In [113]:
modelSFT.predict(x)

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [116]:
meansftmx=CrossValidation(Class_algorithm=modelSFT,x=x,y=modelSFT.true_labels_matrix,metrics_function=accuracy_score,nb_k_fold=4)
np.mean(meansftmx)

algorithm did  converge under 100 iterations (at 65 iterations)
algorithm did  converge under 100 iterations (at 66 iterations)
algorithm did  converge under 100 iterations (at 56 iterations)
algorithm did  converge under 100 iterations (at 62 iterations)


0.671

Further steps:

 -introduce Newton Raphson for softmax regression

 -explore further inference statistics ( log odds, inference for multinomial logistic regressions)

 -do automatic variable selection ( stepwise, backward..., lasso..)
 
 -do real project using all these developped algorithms



In [118]:
modelSFT=LogisticRegression(solver="nr",multiclass="softmax")#dont use nr for the moment
modelSFT.fit(x,y)

ValueError: not done yet,incoming